<a href="https://colab.research.google.com/github/brianmulyadi/Designing-Data-Products/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting dataset

This script is used to interact with the ProPublica API to extract the dataset for congress members and bills. We use the 115th and 117th session of Congress specifically to represent a balance between Republican and Democratic majorities. The 116th session of Congress was excluded as the dataset may be skewed by the Covid-19 pandemic.

In [11]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import requests
from datetime import datetime
from datetime import date

In [12]:
# connect to ProPublica API and test connection
headers={'X-API-Key':''}
url = 'https://api.propublica.org/congress/v1/115/bills/hr7401.json'
response = requests.get(url, headers=headers)
print(response.status_code)


200


### Get Congress member data

In [13]:
# get congress members data
member_url = 'https://api.propublica.org/congress/v1/115/house/members.json'
response = requests.get(member_url, headers=headers)
json = response.json()

# generate empty list to be appended
memberId = []
fullName= []
dateOfBirth = []
gender = []
party = []
leadershipRole = []
dwNominate = []
seniority = []
votesAgainstParty = []

# create for loop to append data frame
for x in json['results'][0]['members']:
  memberId.append(x['id'])
  fullName.append(x['first_name'] + " " + x['last_name'])
  dateOfBirth.append(x['date_of_birth'])
  gender.append(x['gender'])
  party.append(x['party']) 
  leadershipRole.append(x['leadership_role'])
  dwNominate.append(x['dw_nominate'])
  seniority.append(x['seniority'])
  votesAgainstParty.append(x.get('votes_against_party_pct'))

# removed field 'party' from the dataframe due to overlap with sponsorParty from bills API
member_list = list(zip(memberId, fullName, dateOfBirth, gender, leadershipRole, dwNominate, seniority, votesAgainstParty,party))
memberDf = pd.DataFrame(member_list, columns=['memberId', 'fullName', 'dateOfBirth', 'gender', 'leadershipRole', 'dwNominate', 'seniority', 'votesAgainstParty','party'])
memberDf.head()

,memberId,fullName,dateOfBirth,gender,leadershipRole,dwNominate,seniority,votesAgainstParty,party
0,A000374,Ralph Abraham,1954-09-16,M,None,0.508,4,2.44,R
1,A000370,Alma Adams,1946-05-27,F,None,-0.469,6,1.48,D
2,A000055,Robert Aderholt,1965-07-22,M,None,0.361,22,2.42,R
3,A000371,Pete Aguilar,1979-06-19,M,None,-0.285,4,4.81,D
4,A000372,Rick Allen,1951-11-07,M,None,0.611,4,1.43,R


In [14]:
memberDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   memberId           456 non-null    object 
 1   fullName           456 non-null    object 
 2   dateOfBirth        456 non-null    object 
 3   gender             456 non-null    object 
 4   leadershipRole     99 non-null     object 
 5   dwNominate         443 non-null    float64
 6   seniority          456 non-null    object 
 7   votesAgainstParty  450 non-null    float64
 8   party              456 non-null    object 
dtypes: float64(2), object(7)
memory usage: 32.2+ KB


### Clean up data for Congress members

In [15]:
# change dateOfBirth to age
def calculate_age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

memberDf['age'] = memberDf['dateOfBirth'].apply(calculate_age)
del memberDf['dateOfBirth']

memberDf.head()

,memberId,fullName,gender,leadershipRole,dwNominate,seniority,votesAgainstParty,party,age
0,A000374,Ralph Abraham,M,None,0.508,4,2.44,R,68
1,A000370,Alma Adams,F,None,-0.469,6,1.48,D,76
2,A000055,Robert Aderholt,M,None,0.361,22,2.42,R,57
3,A000371,Pete Aguilar,M,None,-0.285,4,4.81,D,43
4,A000372,Rick Allen,M,None,0.611,4,1.43,R,71


In [16]:
# change categorical variables into boolean values
memberDf['leadershipRole'] = np.where(memberDf['leadershipRole'].isnull(), 0, 1)
memberDf['dwNominate'] = memberDf['dwNominate'].fillna(0)
memberDf['votesAgainstParty'] = memberDf['votesAgainstParty'].fillna(0)

memberDf.head()

,memberId,fullName,gender,leadershipRole,dwNominate,seniority,votesAgainstParty,party,age
0,A000374,Ralph Abraham,M,0,0.508,4,2.44,R,68
1,A000370,Alma Adams,F,0,-0.469,6,1.48,D,76
2,A000055,Robert Aderholt,M,0,0.361,22,2.42,R,57
3,A000371,Pete Aguilar,M,0,-0.285,4,4.81,D,43
4,A000372,Rick Allen,M,0,0.611,4,1.43,R,71


### Get Congress bills data

In [17]:
# generate empty dataframe for the bills
billId = []
billTitle= []
policyArea = []
introduceDate = []
memberId = []
sponsorParty = []
sponsorGender = []
sponsorBill = []
sponsorAgainst = []
cosponsors = []
cosponsorsDem = []
cosponsorsRep = []
billStatus = []

# create for loop to get data from the thousands of bills
for x in range(1, 10): # change the range with the total number of bills
  url = 'https://api.propublica.org/congress/v1/117/bills/hr' + str(x) + '.json'
  response = requests.get(url, headers=headers)
  json = response.json()
  billId.append(json['results'][0]['bill_slug'])
  billTitle.append(json['results'][0]['short_title'])
  policyArea.append(json['results'][0]['primary_subject'])
  introduceDate.append(json['results'][0]['introduced_date'])
  memberId.append(json['results'][0]['sponsor_id'])
  sponsorParty.append(json['results'][0]['sponsor_party'])
  cosponsors.append(json['results'][0]['cosponsors'])
  cosponsorsDem.append(json['results'][0]['cosponsors_by_party'].get('D'))
  cosponsorsRep.append(json['results'][0]['cosponsors_by_party'].get('R'))
  billStatus.append(json['results'][0]['house_passage'])
  

main_list = list(zip(billId, billTitle, policyArea, introduceDate, memberId, sponsorParty, cosponsors, cosponsorsDem, cosponsorsRep, billStatus))
df = pd.DataFrame(main_list, columns=['billId', 'billTitle', 'policyArea', 'introduceDate', 'memberId', 'sponsorParty', 'cosponsors', 'cosponsorsDem', 'cosponsorsRep', 'billStatus'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   billId         9 non-null      object 
 1   billTitle      9 non-null      object 
 2   policyArea     9 non-null      object 
 3   introduceDate  9 non-null      object 
 4   memberId       9 non-null      object 
 5   sponsorParty   9 non-null      object 
 6   cosponsors     9 non-null      int64  
 7   cosponsorsDem  7 non-null      float64
 8   cosponsorsRep  2 non-null      float64
 9   billStatus     5 non-null      object 
dtypes: float64(2), int64(1), object(7)
memory usage: 848.0+ bytes


### Cleaning bills data

In [19]:
# filter out rows where member ID is null
df = df[df['memberId']!=""]
df['cosponsorsDem'] = df['cosponsorsDem'].fillna(0) / df['cosponsors']
df['cosponsorsDem'] = df['cosponsorsDem'].fillna(0)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 7
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   billId         7 non-null      object 
 1   billTitle      7 non-null      object 
 2   policyArea     7 non-null      object 
 3   introduceDate  7 non-null      object 
 4   memberId       7 non-null      object 
 5   sponsorParty   7 non-null      object 
 6   cosponsors     7 non-null      int64  
 7   cosponsorsDem  7 non-null      float64
 8   cosponsorsRep  2 non-null      float64
 9   billStatus     5 non-null      object 
dtypes: float64(2), int64(1), object(7)
memory usage: 616.0+ bytes


In [20]:
# change billStatus and issponsorPartyD to boolean
df['billStatus'] = np.where(df['billStatus'].isnull(), 0, 1)
df['issponsorPartyD'] = np.where(df['sponsorParty'] == "D", 1, 0)
del df['sponsorParty']

df.head()

,billId,billTitle,policyArea,introduceDate,memberId,cosponsors,cosponsorsDem,cosponsorsRep,billStatus,issponsorPartyD
0,hr1,For the People Act of 2021,Government Operations and Politics,2021-01-04,S001168,222,1.0,NaN,1,1
2,hr3,Elijah E. Cummings Lower Drug Costs Now Act,Health,2021-04-22,P000034,92,1.0,NaN,0,1
3,hr4,John R. Lewis Voting Rights Advancement Act of...,Government Operations and Politics,2021-08-17,S001185,223,1.0,NaN,1,1
4,hr5,Equality Act,"Civil Rights and Liberties, Minority Issues",2021-02-18,C001084,224,1.0,NaN,0,1
5,hr6,American Dream and Promise Act of 2021,Immigration,2021-03-03,R000486,175,1.0,NaN,1,1


In [21]:
# merge the Congress members dataframe to the bills dataframe
mergeDf = pd.merge(df, memberDf, on='memberId', how='left')
mergeDf.tail()

,billId,billTitle,policyArea,introduceDate,memberId,cosponsors,cosponsorsDem,cosponsorsRep,billStatus,issponsorPartyD,fullName,gender,leadershipRole,dwNominate,seniority,votesAgainstParty,party,age
2,hr4,John R. Lewis Voting Rights Advancement Act of...,Government Operations and Politics,2021-08-17,S001185,223,1.000000,NaN,1,1,Terri Sewell,F,0,-0.390,8,3.67,D,57
3,hr5,Equality Act,"Civil Rights and Liberties, Minority Issues",2021-02-18,C001084,224,1.000000,NaN,0,1,David Cicilline,M,0,-0.390,8,2.34,D,61
4,hr6,American Dream and Promise Act of 2021,Immigration,2021-03-03,R000486,175,1.000000,NaN,1,1,Lucille Roybal-Allard,F,0,-0.471,26,1.74,D,81
5,hr7,Paycheck Fairness Act,Labor and Employment,2021-01-28,D000216,225,0.986667,3.0,1,1,Rosa DeLauro,F,0,-0.416,28,2.36,D,79
6,hr8,Bipartisan Background Checks Act of 2021,Crime and Law Enforcement,2021-03-01,T000460,210,0.985714,3.0,1,1,Mike Thompson,M,0,-0.396,20,3.73,D,71


### Preparing data for models

In [22]:
# select relevant columns
finalDf = mergeDf[['policyArea','issponsorPartyD','cosponsors','cosponsorsDem','billStatus','leadershipRole','dwNominate','seniority','votesAgainstParty']]
finalDf.head()

,policyArea,issponsorPartyD,cosponsors,cosponsorsDem,billStatus,leadershipRole,dwNominate,seniority,votesAgainstParty
0,Government Operations and Politics,1,222,1.0,1,0,-0.469,12,1.97
1,Health,1,92,1.0,0,0,-0.404,32,2.97
2,Government Operations and Politics,1,223,1.0,1,0,-0.390,8,3.67
3,"Civil Rights and Liberties, Minority Issues",1,224,1.0,0,0,-0.390,8,2.34
4,Immigration,1,175,1.0,1,0,-0.471,26,1.74


In [23]:
# create dummy columns for policyArea
finalDf = pd.get_dummies(finalDf,columns=['policyArea'])
finalDf.head()

,issponsorPartyD,cosponsors,cosponsorsDem,billStatus,leadershipRole,dwNominate,seniority,votesAgainstParty,"policyArea_Civil Rights and Liberties, Minority Issues",policyArea_Crime and Law Enforcement,policyArea_Government Operations and Politics,policyArea_Health,policyArea_Immigration,policyArea_Labor and Employment
0,1,222,1.0,1,0,-0.469,12,1.97,0,0,1,0,0,0
1,1,92,1.0,0,0,-0.404,32,2.97,0,0,0,1,0,0
2,1,223,1.0,1,0,-0.390,8,3.67,0,0,1,0,0,0
3,1,224,1.0,0,0,-0.390,8,2.34,1,0,0,0,0,0
4,1,175,1.0,1,0,-0.471,26,1.74,0,0,0,0,1,0


In [24]:
finalDf.to_csv('dataset.csv')